In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
from PIL import Image
import Butterfly_identification.preprocessbutterfly as preproc
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.metrics import CategoricalAccuracy

from tensorflow.keras.applications.vgg16 import preprocess_input
# from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
from os.path import join, sep
from tensorflow.data import Dataset, AUTOTUNE
from tensorflow.strings import split
from tensorflow.io import decode_jpeg, read_file
from tensorflow import argmax
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential, layers
import gc
from tensorflow.compat.v1 import ConfigProto , Session, InteractiveSession
from tensorflow.compat.v1.keras.backend import set_session

In [4]:
from tensorflow.data.experimental import cardinality
from IPython.display import Audio
from IPython.display import Audio
sound_file = 'treasure.mp3' #un son dans votre ordi ou une url


In [5]:
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
set_session(Session(config=config));
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

2021-08-31 09:17:30.271607: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-31 09:17:30.343860: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 09:17:30.365460: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 09:17:30.366246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

# Get Data

In [43]:
df_train,df_val,df_test = preproc.get_data()

In [44]:
df_train.shape

(36536, 15)

In [45]:
df_train = preproc.feature_engineering(df_train)
df_val = preproc.feature_engineering(df_val)
df_test = preproc.feature_engineering(df_test)

## Generator implementation

In [46]:
PATH_TRAIN = '/home/zac/code/Em3line/Butterfly_identification/raw_data/IGM_labels/Train/'
PATH_VAL = '/home/zac/code/Em3line/Butterfly_identification/raw_data/IGM_labels/Val/'
PATH_TEST = '/home/zac/code/Em3line/Butterfly_identification/raw_data/IGM_labels/Test/'

In [47]:
#df_train = preproc.get_data_minphoto(df_train)
df_train = preproc.resampling(df_train)

In [48]:
df_val = preproc.filter_val_test(df_train, df_val)
df_test = preproc.filter_val_test(df_train, df_test)

In [49]:
df_train['full_path'] = df_train['species']+'/'+df_train['image_name']
df_val['full_path'] = df_val['species']+'/'+df_val['image_name']
df_test['full_path'] = df_test['species']+'/'+df_test['image_name']

In [50]:
black_list = list(PATH_TRAIN + df_train['full_path'])
black_list_val = list(PATH_VAL + df_val['full_path'])
black_list_test = list(PATH_TEST + df_test['full_path'])

In [51]:
train_names = df_train['species'].drop_duplicates()

In [54]:
df_train

,author,barcode,country,dec_lat,dec_long,family,genus,image_name,image_path,infraspecific_epithet,primary_division,specific_epithet,subfamily,subspecific_epithet,token,path_to_image,species,full_path
7054c858-89e7-4288-8f63-d9b49e53c394,"Fruhstorfer, 1906",ETHZ-ENT0119779,France,46.4864,6.0625,Papilionidae,Parnassius,ETHZ_ENT01_2018_03_19_136064.JPG,2018_03_19R,None,Bourgogne-Franche-Comté,apollo,Parnassiinae,nivatus,7054c858-89e7-4288-8f63-d9b49e53c394,../raw_data/IMG/ETHZ_ENT01_2018_03_19_136064.JPG,Parnassius_apollo,Parnassius_apollo/ETHZ_ENT01_2018_03_19_136064...
4f354c85-954a-4bd9-9a1b-3ab603b55bc2,"(Linnaeus, 1758)",ETHZ-ENT0014820,Switzerland,47.412495,8.592223,Pieridae,Pieris,ETHZ_ENT01_2017_04_07_012909.JPG,2017_04_07R,None,Zurich,napi,Pierinae,None,4f354c85-954a-4bd9-9a1b-3ab603b55bc2,../raw_data/IMG/ETHZ_ENT01_2017_04_07_012909.JPG,Pieris_napi,Pieris_napi/ETHZ_ENT01_2017_04_07_012909.JPG
56c54b5c-2d3f-410c-9df8-895f5f16e55e,"(Rottemburg, 1775)",ETHZ-ENT0026790,Switzerland,47.36591,8.35644,Lycaenidae,Polyommatus,ETHZ_ENT01_2017_05_08_026461.JPG,2017_05_08R,None,Aargau,icarus,Polyommatinae,None,56c54b5c-2d3f-410c-9df8-895f5f16e55e,../raw_data/IMG/ETHZ_ENT01_2017_05_08_026461.JPG,Polyommatus_icarus,Polyommatus_icarus/ETHZ_ENT01_2017_05_08_02646...
e611e40d-d19c-463b-b4ea-1f86886a65de,"(Linnaeus, 1761)",ETHZ-ENT0022428,Switzerland,46.25785,7.54104,Lycaenidae,Plebejus,ETHZ_ENT01_2017_04_28_021611.JPG,2017_04_28R,None,Valais,idas,Polyommatinae,None,e611e40d-d19c-463b-b4ea-1f86886a65de,../raw_data/IMG/ETHZ_ENT01_2017_04_28_021611.JPG,Plebejus_idas,Plebejus_idas/ETHZ_ENT01_2017_04_28_021611.JPG
47834000-94aa-4910-a5ac-5b6c15850847,"(Rottemburg, 1775)",ETHZ-ENT0013616,Switzerland,46.02629,8.90882,Nymphalidae,Melitaea,ETHZ_ENT01_2017_04_06_012709.JPG,2017_04_06R,None,Ticino,athalia,Nymphalinae,None,47834000-94aa-4910-a5ac-5b6c15850847,../raw_data/IMG/ETHZ_ENT01_2017_04_06_012709.JPG,Melitaea_athalia,Melitaea_athalia/ETHZ_ENT01_2017_04_06_012709.JPG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
e6920036-f1d3-47da-9549-cdaed0591d0b,"(Linnaeus, 1763)",ETHZ-ENT0028876,Japan,None,None,Nymphalidae,Nymphalis,ETHZ_ENT01_2017_05_11_028859.JPG,2017_05_11R,None,None,canace,Nymphalinae,None,e6920036-f1d3-47da-9549-cdaed0591d0b,../raw_data/IMG/ETHZ_ENT01_2017_05_11_028859.JPG,Nymphalis_canace,Nymphalis_canace/ETHZ_ENT01_2017_05_11_028859.JPG
e93e05a0-6b56-42ce-8660-88534293c0d9,"Oberthür, 1876",ETHZ-ENT0016036,China,32.131936,119.078137,Pieridae,Anthocharis,ETHZ_ENT01_2017_04_10_014280.JPG,2017_04_10R,None,Jiangsu,bambusarum,Pierinae,None,e93e05a0-6b56-42ce-8660-88534293c0d9,../raw_data/IMG/ETHZ_ENT01_2017_04_10_014280.JPG,Anthocharis_bambusarum,Anthocharis_bambusarum/ETHZ_ENT01_2017_04_10_0...
ea18eec5-c4cc-475e-b17e-7ff963693601,"(Kindermann, 1853)",ETHZ-ENT0117097,Russia,50.632499,87.022384,Papilionidae,Parnassius,ETHZ_ENT01_2018_03_08_132516.JPG,2018_03_08R,None,Altai,ariadne,Parnassiinae,ariadne,ea18eec5-c4cc-475e-b17e-7ff963693601,../raw_data/IMG/ETHZ_ENT01_2018_03_08_132516.JPG,Parnassius_ariadne,Parnassius_ariadne/ETHZ_ENT01_2018_03_08_13251...
ecb10995-6293-46d2-b0a9-3e2e2afa828e,"Géné, 1839",ETHZ-ENT0007658,France,41.8082,9.1763,Papilionidae,Papilio,ETHZ_ENT01_2017_03_20_004671.JPG,2017_03_20R,None,Corsica,hospiton,Papilioninae,None,ecb10995-6293-46d2-b0a9-3e2e2afa828e,../raw_data/IMG/ETHZ_ENT01_2017_03_20_004671.JPG,Papilio_hospiton,Papilio_hospiton/ETHZ_ENT01_2017_03_20_004671.JPG


In [15]:
test_predict = df_train['full_path'][0]

In [16]:
del df_train
del df_val
del df_test

In [17]:
filtered_train = [join(PATH_TRAIN, species) for species in black_list]
filtered_val = [join(PATH_VAL, species) for species in black_list_val]
filtered_test = [join(PATH_TEST, species) for species in black_list_test]

In [18]:
train_ds = Dataset.from_tensor_slices(filtered_train)
val_ds = Dataset.from_tensor_slices(filtered_val)
test_ds = Dataset.from_tensor_slices(filtered_test)

2021-08-31 09:17:42.949639: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 09:17:42.950225: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 09:17:42.950682: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 09:17:42.951617: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-08-31 09:17:42.952104: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [19]:
del filtered_train
del filtered_test
del filtered_val

In [20]:
def get_label(file_path,class_names=train_names):
  # convert the path to a list of path components
  parts = split(file_path, sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return argmax(one_hot)

In [21]:
def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = decode_jpeg(img, channels=3)
    img = preprocess_input(img)
    # resize the image to the desired size
    return img


In [22]:
def process_path(file_path, class_names=train_names):
    label = get_label(file_path, class_names)
    # load the raw data from the file as a string
    img = read_file(file_path)
    img = decode_img(img)
    return img, label

In [23]:
AUTOTUNE = AUTOTUNE

In [24]:
train_ds = train_ds.map(process_path, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(process_path, num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [25]:
def configure_for_performance(ds):
    #ds = ds.cache()
    ds = ds.shuffle(buffer_size=1000)
    ds = ds.batch(batch_size=32)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_ds = configure_for_performance(train_ds)
val_ds = configure_for_performance(val_ds)
test_ds = configure_for_performance(test_ds)

In [26]:
type(train_ds)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

# Data Augmentation

In [27]:
data_augmentation = Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [28]:
IMG_SIZE = 128

resize_and_rescale = Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])


In [29]:
train_ds

<PrefetchDataset shapes: ((None, None, None, 3), (None,)), types: (tf.uint8, tf.int64)>

#augmentation = ImageDataGenerator(rotation_range=25, width_shift_range=0.2,
                  height_shift_range=0.2,  
                  zoom_range=0.1, horizontal_flip=True)

In [30]:
batch_size = 32
def prepare(ds, shuffle=False, augment=False):
    # Resize and rescale all datasets
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

    # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

    # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)


In [31]:
#train_ds = prepare(train_ds, shuffle=False, augment=True)

image_batch, label_batch = next(iter(train_ds))
print(len(label_batch))
plt.figure(figsize=(10, 10))
for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    print(image_batch.shape)
    plt.imshow(image_batch[i].numpy().astype("uint8"))
    label = label_batch[i]
    plt.title(train_names[label])
    plt.axis("off")

In [32]:
#train_ds = train_ds.map(augmentation.flow, num_parallel_calls=AUTOTUNE)

In [33]:
# y_train = list(map(lambda x: x[1].numpy(), train_ds))

In [34]:
# pd.DataFrame(y_train).max()

In [35]:
# y_train

# Model

In [36]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras import optimizers, layers, models
from tensorflow.keras.callbacks import EarlyStopping

In [37]:
def model_VGG16(train_ds, val_ds, test_ds, image_size=(128, 128, 3),  patience=3, learning_rate=0.001, nb_epochs=15, nb_couches_dense_layer=130):
    
    model = VGG16(weights="imagenet", include_top=False, input_shape = image_size)
    
    



    model.trainable = False
    
    #add last layers
    
    flatten_layer = layers.Flatten()
    dense_layer = layers.Dense(nb_couches_dense_layer, activation='relu')
    '''dropout_layer1 = layers.Dropout(.2, input_shape=(2,))
    dense_layer2 = layers.Dense(1000, activation='relu')
    dropout_layer2 = layers.Dropout(.2, input_shape=(2,))'''
    prediction_layer = layers.Dense(228, activation='softmax')
    model_final = models.Sequential([
        data_augmentation,
        resize_and_rescale,
        model,
        flatten_layer,
        dense_layer,
        prediction_layer
    ])
    
    #build model
    opt = optimizers.Adam(learning_rate=learning_rate)
    model_final.compile(loss='sparse_categorical_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    

    
    #set earlystopping
    es = EarlyStopping(monitor='val_loss', patience=patience, verbose=1, restore_best_weights=True)
    
    #launch model
    history = model_final.fit(train_ds, 
                    validation_data=val_ds, 
                    epochs=nb_epochs,  
                    callbacks=[es])
    
    model_final.summary()
    
    #evaluate model
    res_vgg = model_final.evaluate(test_ds)
    
    test_accuracy_vgg = res_vgg[-1]
    
    
    return (f"test_accuracy_vgg = {round(test_accuracy_vgg,2)*100} %"), model_final

In [38]:
from Butterfly_identification.trainer import plot_history

In [39]:
check, model = model_VGG16(train_ds, val_ds, test_ds, nb_epochs=50, nb_couches_dense_layer=100)
Audio(sound_file, autoplay=True)

Epoch 1/50


2021-08-31 09:17:46.476092: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2021-08-31 09:17:52.794868: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8202
2021-08-31 09:18:05.012189: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.04GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-31 09:18:06.138739: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.55GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-31 09:18:07.115490: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying t

22/23 [===========================>..] - ETA: 0s - loss: nan - accuracy: 0.0014

2021-08-31 09:18:11.558714: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-31 09:18:11.664933: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.06GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-31 09:18:11.757753: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.09GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


23/23 [==============================] - ETA: 0s - loss: nan - accuracy: 0.0014

2021-08-31 09:18:11.857807: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.15GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-31 09:18:30.372426: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.82GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2021-08-31 09:18:31.196376: W tensorflow/core/common_runtime/bfc_allocator.cc:272] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.44GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.


23/23 [==============================] - 47s 1s/step - loss: nan - accuracy: 0.0014 - val_loss: nan - val_accuracy: 0.0281
Epoch 2/50


KeyboardInterrupt: 

In [ ]:
from PIL import Image
test_predict = Image.open(PATH_TRAIN+test_predict)
model.predict(preprocess_input(np.expand_dims(np.array(test_predict), axis=0)))
# PATH_TRAIN+test_predict